# COSINE SIMILARITY
- plot DATA

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
# import plotly.express as px

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

from sklearn.feature_extraction.text import TfidfVectorizer
# import spacy
# from spacy.lang.en.stop_words import STOP_WORDS
# nlp = spacy.load('en_core_web_sm')
import difflib
from sklearn.metrics.pairwise import cosine_similarity
sns.set()
warnings.filterwarnings("ignore")

In [2]:
movies = pd.read_csv('./film_festival_ver3.csv')
movies.head(-5)

,year,film_festival,title,director,field,award,genre,plot,rating,runtime,prodYear,actors,img_link,stills
0,2021,소리아 국제단편영화제,조지아,제이 박,NaN,NaN,"가족,범죄",병들고 가난한 부부가 딸의 억울한 죽음을 규명하기 위해 나선다. 가해자를 밝히지도 ...,12세관람가,29.0,2020.0,"이양희,이채경,김규남",https://ssl.pstatic.net/imgmovie/mdi/mit110/19...,http://file.koreafilm.or.kr/thm/01/copy/00/55/...
1,2021,보고타 단편영화제,미미(美味): 궁극의 식사,한재원,경쟁부문,NaN,"코메디,공포",미각을 잃은 미식 유튜버 성호는 어딘가 기괴한 음식점에 방문한다.,NaN,19.0,2020.0,"김최용준,정경임,이종윤",https://ssl.pstatic.net/imgmovie/mdi/mit110/20...,http://file.koreafilm.or.kr/thm/01/copy/00/59/...
2,2021,엑상프로방스 국제단편영화제,아이즈앤혼즈,임채린,International Competition of Experimental Shorts,NaN,기타,NaN,NaN,NaN,NaN,NaN,https://ssl.pstatic.net/imgmovie/mdi/mit110/20...,NaN
3,2021,서울독립영화제,절해고도,김미영,페스티벌 초이스,NaN,드라마,절해고도. '육지에서 멀리 떨어진 바다의 외딴 섬'이라는 뜻을 지닌 제목은 삶의 표...,NaN,116.0,2021.0,"박종환,이연,강경헌",https://ssl.pstatic.net/imgmovie/mdi/mit110/20...,http://file.koreafilm.or.kr/thm/01/copy/00/59/...
4,2021,서울독립영화제,장갑을 사러,이현주,단편경쟁,NaN,멜로/로맨스,피아노를 가르치는 인경은 연인의 일본 발령 소식에 일본어 학원을 다니기 시작한다. ...,NaN,29.0,2021.0,"김정민,양종욱,김하람",https://ssl.pstatic.net/imgmovie/mdi/mit110/20...,http://file.koreafilm.or.kr/thm/01/copy/00/59/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3352,2000,정동진독립영화제,심청,유종미,단편과 애니 2,NaN,드라마,"딸을 극단적으로 사랑하는 눈먼 아버지, 무당이었던 어머니가 그리운 눈먼 아버지의 딸...",NaN,30.0,2000.0,"백은경,유순철,이동현",https://ssl.pstatic.net/imgmovie/mdi/mit110/02...,http://file.koreafilm.or.kr/thm/01/copy/00/31/...
3353,2000,정동진독립영화제,1979년 10월 28일 일요일 맑음,권종관,재밌는 단편모음 2,NaN,드라마,"1979년 10월 28일 일요일, 영환의 부모님은 아침 일찍 시골에 내려가신다. 엄...",NaN,17.0,1999.0,"강혜은,박진아,이동근",https://ssl.pstatic.net/imgmovie/mdi/mit110/03...,http://file.koreafilm.or.kr/thm/01/copy/00/36/...
3354,2000,정동진독립영화제,81 해적 디스코 왕이 되다,김동원,단편과 애니 2,NaN,기타,NaN,NaN,NaN,NaN,NaN,https://ssl.pstatic.net/imgmovie/mdi/mit110/03...,NaN
3355,2000,정동진독립영화제,미메시스 TV - 에피소드 1,전승일,단편과 애니 2,NaN,드라마,사람이 컴퓨터 앞에 앉아 있다. 밖에는 디스켓이 날아다니고 그는 무료함을 느낀다. ...,NaN,3.0,2000.0,NaN,https://ssl.pstatic.net/imgmovie/mdi/mit110/04...,http://file.koreafilm.or.kr/thm/01/copy/00/31/...


In [7]:
# 영화 제목 + 줄거리

X = movies['title'] + " " + movies['plot']
X

0       조지아 병들고 가난한 부부가 딸의 억울한 죽음을 규명하기 위해 나선다. 가해자를 밝...
1       미미(美味): 궁극의 식사 미각을 잃은 미식 유튜버 성호는 어딘가 기괴한 음식점에 ...
2                                                     NaN
3       절해고도 절해고도. '육지에서 멀리 떨어진 바다의 외딴 섬'이라는 뜻을 지닌 제목은...
4       장갑을 사러 피아노를 가르치는 인경은 연인의 일본 발령 소식에 일본어 학원을 다니기...
                              ...                        
3357    엔조이 유어 썸머 어느 무더운 여름, 아직 회사 생활에 적응하지 못한 한 남자에게 ...
3358    너무 많이 본 사나이 승수는 앞집에 세들어 사는 여자를 몰래 카메라에 담기 위해 친...
3359    아리랑 아리랑 음악에 맞추어 한국 전통 문양들이 생성되고 소멸된다. 연출의도_ 관성...
3360    온실 작은 아파트에 홀로 사는 노인은 빨래, 신문 읽기, 라디오 듣기, 노래 따라 ...
3361    초촌면 신암리 삭막해진 농촌에서 유기농사를 짓는 주인공은 고집스러움 때문에 마을 사...
Length: 3362, dtype: object

In [8]:
len(X)

3362

## Vectorizer *****
- plot DATA => 3055 개

In [9]:
Vectorizer = TfidfVectorizer() #??

In [11]:
# np.nan is an invalid document, expected byte or unicode string.
X_na = X.dropna(axis=0)
X_na

0       조지아 병들고 가난한 부부가 딸의 억울한 죽음을 규명하기 위해 나선다. 가해자를 밝...
1       미미(美味): 궁극의 식사 미각을 잃은 미식 유튜버 성호는 어딘가 기괴한 음식점에 ...
3       절해고도 절해고도. '육지에서 멀리 떨어진 바다의 외딴 섬'이라는 뜻을 지닌 제목은...
4       장갑을 사러 피아노를 가르치는 인경은 연인의 일본 발령 소식에 일본어 학원을 다니기...
8       내일의 연인들 29세의 대학원생 정안은 어느 날, 선애 누나의 전화를 받는다. 선애...
                              ...                        
3357    엔조이 유어 썸머 어느 무더운 여름, 아직 회사 생활에 적응하지 못한 한 남자에게 ...
3358    너무 많이 본 사나이 승수는 앞집에 세들어 사는 여자를 몰래 카메라에 담기 위해 친...
3359    아리랑 아리랑 음악에 맞추어 한국 전통 문양들이 생성되고 소멸된다. 연출의도_ 관성...
3360    온실 작은 아파트에 홀로 사는 노인은 빨래, 신문 읽기, 라디오 듣기, 노래 따라 ...
3361    초촌면 신암리 삭막해진 농촌에서 유기농사를 짓는 주인공은 고집스러움 때문에 마을 사...
Length: 3055, dtype: object

In [12]:
len(X_na)

3055

In [15]:
# Plot NaN 개수
X_cnt_nan = len(X) - len(X_na)
X_cnt_nan

307

In [16]:
X_cnt_nan / len(X)

0.09131469363474122

In [13]:
X_vec = Vectorizer.fit_transform(X_na)
X_vec

<3055x44992 sparse matrix of type '<class 'numpy.float64'>'
	with 128348 stored elements in Compressed Sparse Row format>

In [17]:
X_vec = X_vec.toarray()
print(X_vec.shape)
X_vec

(3055, 44992)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Content Similarity
- 모듈 사용. 연산 오래 걸림.
- 뭔가 나옴. 이해.

In [18]:
similarity = cosine_similarity(X_vec)
similarity

array([[1.        , 0.        , 0.00289128, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00289128, 0.        , 1.        , ..., 0.        , 0.01295611,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.00327306,
        0.        ],
       [0.        , 0.        , 0.01295611, ..., 0.00327306, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [19]:
similarity.shape

(3055, 3055)

## Recommendation Function
- 값이 나온다.
- 작동 원리를 구조적으로 알아야 한다.

### Similarity 영화
- categorical DATA == TOP10

In [75]:
def recommendContent(movie, movies=list(movies['title'])):
    title = difflib.get_close_matches(movie, movies)[0]
    index = movies.index(title)
    
    similar_movies = similarity[index]

    similar_movies = sorted(range(len(similar_movies)), key=lambda k: similar_movies[k], reverse = True)[1:11] # sorting Top 10
    recommended = [movies[i] for i in similar_movies]
    
    
    return recommended

In [77]:
# TEST 1
recommendContent("절해고도")

['나들이',
 '별주부',
 '삭제하시겠습니까?',
 '앞서는 마음',
 '이상한 나라의 김민수',
 '나리 대 나리',
 '배고픈 하루',
 '알 수 없는 슬픔이 있어',
 '국물은 공짜가 아니다',
 '된장']

In [22]:
# TEST 2
recommendContent("내일의 연인들")

['웰컴',
 '부탁',
 '블랙아웃',
 '밀실',
 '치석',
 '겨울의 피아니스트',
 '지옥화',
 '급식',
 '괜찮아, 임마',
 '텐더 앤 윗치']

### Similarity 유사도
- 수치 데이터 (정량적)

In [41]:
len(similarity)

3055

In [45]:
similarity[0]

array([1.        , 0.        , 0.00289128, ..., 0.        , 0.        ,
       0.        ])

In [71]:
def recommendContentScore(movie, movies=list(movies['title'])):
    title = difflib.get_close_matches(movie, movies)[0]
    index = movies.index(title)
    
    similar_movies = similarity[index]

    similar_movies_num = sorted(range(len(similar_movies)), key=lambda k: similar_movies[k], reverse = True)[1:11] # sorting Top 10
    recommended = [similar_movies[i] for i in similar_movies_num]

    print(np.shape(similar_movies))
    return recommended

In [72]:
# TEST1
recommendContentScore("절해고도")

(3055,)


[0.06551848974290087,
 0.06476437814160385,
 0.06355470456680348,
 0.05967191710835367,
 0.05062586971826823,
 0.049295568947235226,
 0.045045446767725254,
 0.04357244257399216,
 0.04050765009058096,
 0.04044706417083744]